In [5]:
# Import source

cd("../Source")
include("ChiralMC.jl")
using ChiralMC

include("functions_msc.jl")
include("functions_observables.jl")
include("functions_parallel.jl")

using Base.Test
using Plots
gr()
using Distributions
cd("../Notebooks/")

In [6]:
@everywhere const two_pi = 2π

# We run a simulation with the parameters
g = 0.3    # Gauge coupling
ν = 0.3    # Anisotropy
H = -0.72   # External field

# Other parameters
L = 32     # System length
T = 0.4   # Temperature
γ = 1.0    # Order parameter amplitud

# Calculate periodic boundary conditioned f s.t. fL ∈ N
f = ceil(abs(H/(2π)*L))/L*sign(H)
# Calculate inverse temperature
β = 1/T

# Create system
syst = SystConstants(L, γ, 1/g^2, ν, f, β)
sim = Controls(π/3, 0.4, 3.0)

# Measurement parameters
M = 300    # Number of measurements
Δt = 200  # Interval between measurements (MCS)
# Construct k-matrix where the horizontal axis contains kx ∈ [-π, π), while
# the vertical axis contain ky ∈ [-π, π) at the second component
k_matrix = [[2π/L*(x-1-L/2), 2π/L*(L/2-y)] for y=1:L, x=1:L];

# Motivation

Since we so far don't have parallelized the mc-update, then much time is wasted thermalizing the system since this has to be done on a single cpu. Therefore it is better to use a cpu with higher frequency than what is found on Vilje and other supercomputers. The plan is to have a julia script prepare a folder for each system we want to measure. The folder should contain a file giving the system parameters as before, as well as two saved states that are the final states after thermalizing. As well as a file giving the thermalization time. This will also eliminate the need to re-equilibrate a system that is already equilibrated where we want to do new measurements.

### Moving to work directory

In [7]:
cd("../Data/")

In [10]:
ψ = State(2, syst)
save(ψ, "temp_state")

Saving state to file temp_state


1

In [24]:
function readState(filename::AbstractString)
    open(filename, "r") do file
        line = readline(file)
        if line != "state start"
            throw(DomainError())
        end
        L = parse(Int64, readline(file))
        γ = parse(Float64, readline(file))
        g⁻² = parse(Float64, readline(file))
        ν = parse(Float64, readline(file))
        f = parse(Float64, readline(file))
        β = parse(Float64, readline(file))
        syst = SystConstants(L, γ, g⁻², ν, f, β)
        
        lattice = Array{LatticeSite, 2}(L,L)
        for h_pos = 1:L, v_pos = 1:L
            ϕ_values = split(readline(file), ":")
            A₁ = parse(Float64, ϕ_values[1])
            A₂ = parse(Float64, ϕ_values[2])
            θ⁺ = parse(Float64, ϕ_values[3])
            θ⁻ = parse(Float64, ϕ_values[4])
            u⁺ = parse(Float64, ϕ_values[5])
            u⁻ = parse(Float64, ϕ_values[6])
            lattice[v_pos,h_pos] = LatticeSite([A₁, A₂], θ⁺, θ⁻, u⁺, u⁻)
        end
        
        nbl = latticeNeighbors(lattice,L)
        nnbl = latticeNextNeighbors(lattice,L)
        nnnbl = latticeNNNeighbors(lattice,L)
        
        return State(lattice, syst, nbl, nnbl, nnnbl)
    end
end

readState (generic function with 1 method)

In [28]:
import Base.==

In [31]:
(true &&
false && true)

false

In [35]:
function ==(ϕ₁::LatticeSite, ϕ₂::LatticeSite)
    return (ϕ₁.A[1] == ϕ₂.A[1] && ϕ₁.A[2] == ϕ₂.A[2] && ϕ₁.θ⁺ == ϕ₂.θ⁺ && 
        ϕ₁.θ⁻ == ϕ₂.θ⁻ && ϕ₁.u⁺ == ϕ₂.u⁺ && ϕ₁.u⁻ == ϕ₂.u⁻)
end
function ==(c₁::SystConstants, c₂::SystConstants)
    return (c₁.L == c₂.L && c₁.γ == c₂.γ && c₁.g⁻² == c₂.g⁻² && c₁.ν == c₂.ν && c₁.f == c₂.f && c₁.β == c₂.β)
end

== (generic function with 174 methods)

In [36]:
function ==(ψ₁::State, ψ₂::State)
    check = ψ₁.consts == ψ₂.consts
    L = ψ₁.consts.L
    for h_pos=1:L, v_pos=1:L
        check = check && ψ₁.lattice[v_pos,h_pos] == ψ₂.lattice[v_pos,h_pos]
    end
    return check
end

== (generic function with 174 methods)

In [25]:
ψ_read = readState("temp_state")

ChiralMC.State(ChiralMC.LatticeSite[ChiralMC.LatticeSite([-204.292, 251.008], 4.0679, 2.8203, 0.879275, 0.476315) ChiralMC.LatticeSite([930.374, 1001.8], 2.23182, 1.62128, 0.123718, 0.992317) … ChiralMC.LatticeSite([602.035, 209.562], 4.54858, 5.12353, 0.591308, 0.806446) ChiralMC.LatticeSite([501.753, -68.7539], 1.78959, 0.364271, 0.343379, 0.939197); ChiralMC.LatticeSite([525.745, 281.104], 1.32093, 5.49295, 0.150915, 0.988547) ChiralMC.LatticeSite([-242.032, -387.028], 2.03045, 2.97377, 0.350811, 0.936446) … ChiralMC.LatticeSite([241.999, -900.811], 4.88932, 1.39726, 0.421664, 0.906752) ChiralMC.LatticeSite([-543.752, -905.355], 5.3743, 0.69171, 0.938925, 0.344121); … ; ChiralMC.LatticeSite([821.696, -715.093], 1.68815, 4.8587, 0.393203, 0.919452) ChiralMC.LatticeSite([802.22, 65.5285], 3.31287, 4.15757, 0.076888, 0.99704) … ChiralMC.LatticeSite([413.625, -358.161], 1.97966, 1.40614, 0.198675, 0.980065) ChiralMC.LatticeSite([466.39, 381.256], 6.08097, 5.06127, 0.86525, 0.501341); Ch

In [37]:
ψ == ψ_read

true

In [38]:
ψ_test = State(2, syst)
ψ == ψ_test

false

In [23]:
str = "-204.29223948220715:251.00763062155238:4.067901401957929:2.820301520153933:0.879274862870894:0.47631472318559565"
split(str, ":")

6-element Array{SubString{String},1}:
 "-204.29223948220715"
 "251.00763062155238" 
 "4.067901401957929"  
 "2.820301520153933"  
 "0.879274862870894"  
 "0.47631472318559565"